In [1]:
#Step 1: Data Preparation
#1.1 Handling Missing Data

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
# Load the data
data = pd.read_csv('../data/cleaned_data.csv')

In [3]:
# Convert data types to more memory-efficient formats
for col in data.select_dtypes(include=['float64']).columns:
    data[col] = pd.to_numeric(data[col], downcast='float')

for col in data.select_dtypes(include=['int64']).columns:
    data[col] = pd.to_numeric(data[col], downcast='integer')

## 1.2 Feature Engineering

In [4]:
# Convert TransactionMonth to datetime and extract relevant features
data['TransactionMonth'] = pd.to_datetime(data['TransactionMonth'], errors='coerce')
data['TransactionYear'] = data['TransactionMonth'].dt.year
data['TransactionMonth'] = data['TransactionMonth'].dt.month

# Feature Engineering
data['VehicleAge'] = 2024 - data['RegistrationYear']
data['ProfitMarginRatio'] = data['TotalPremium'] - data['TotalClaims']
data['ProfitMarginRatio'] = data['ProfitMarginRatio'].replace([np.inf, -np.inf], 0)
data['ProfitMarginRatio'] = data['ProfitMarginRatio'].fillna(0)

In [5]:
# Drop irrelevant or non-numeric columns before encoding
data.drop(['UnderwrittenCoverID', 'PolicyID', 'Country', 'MainCrestaZone', 'SubCrestaZone',
           'make', 'Model', 'bodytype', 'Title', 'Language', 'Bank', 'AccountType', 'LegalType', 'MaritalStatus'], axis=1, inplace=True)

## 1.3 Encoding Categorical Data

In [6]:
# Encoding Categorical Data
categorical_features = ['Province', 'PostalCode', 'Gender', 'VehicleType', 'CoverType']

In [7]:
# Reduce cardinality by grouping infrequent categories (if necessary)
for feature in categorical_features:
    top_categories = data[feature].value_counts().nlargest(10).index
    data[feature] = np.where(data[feature].isin(top_categories), data[feature], 'Other')

In [8]:
# One-hot encoding with memory efficiency
data = pd.get_dummies(data, columns=categorical_features, drop_first=True)

# Ensure all columns are numeric and handle non-numeric values
data = data.apply(pd.to_numeric, errors='coerce').fillna(0)

## 1.4 Train-Test Split

In [9]:
# Train-Test Split
X = data.drop(['TotalPremium', 'TotalClaims'], axis=1)
y = data['TotalPremium']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Feature Scaling
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [10]:
# Verify the transformations
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Sample feature names: {X.columns[:5]}")

X_train shape: (699682, 66)
X_test shape: (299864, 66)
Sample feature names: Index(['TransactionMonth', 'IsVATRegistered', 'Citizenship', 'ItemType',
       'mmcode'],
      dtype='object')


# Step 2: Modeling Techniques

## 2.1 Linear Regression


In [11]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Linear Regression Model
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)
y_pred_lr = lr_model.predict(X_test)

# Evaluation
mse_lr = mean_squared_error(y_test, y_pred_lr)
r2_lr = r2_score(y_test, y_pred_lr)
print(f"Linear Regression - MSE: {mse_lr}, R2: {r2_lr}")

Linear Regression - MSE: 11071.161845482868, R2: 0.5517246013524777


## 2.2 Random Forest

In [18]:
from sklearn.ensemble import RandomForestRegressor

# Random Forest Model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Evaluation
mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)
print(f"Random Forest - MSE: {mse_rf}, R2: {r2_rf}")

## 2.3 XGBoost

In [17]:
import xgboost as xgb

# XGBoost Model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, random_state=42)
xgb_model.fit(X_train, y_train)
y_pred_xgb = xgb_model.predict(X_test)

# Evaluation
mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)
print(f"XGBoost - MSE: {mse_xgb}, R2: {r2_xgb}")

XGBoostError: bad allocation